In [1]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline  

matplotlib.style.use('ggplot')

import numpy as np
import pandas as pd
import random
import skflow
import tensorflow as tf

from sklearn import datasets, cross_validation, metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [2]:
data = pd.read_csv('train.csv')

for col in ['DayOfWeek','PdDistrict']:
    dummies = pd.get_dummies(data[col])
    data[col[0:3]+"_"+dummies.columns] = dummies

cat_dict = {}
for col in ['Category','DayOfWeek','PdDistrict']:
    s = pd.Series(data[col], dtype="category")
    cat_dict[col] = s.cat.categories
    s.cat.categories = [int(num) for num in range(0,len(s.cat.categories))]
    data[col[0:3]+'Num'] = pd.Series(s, dtype="int")

data[['X','Y']] = preprocessing.normalize(data[['X','Y']], norm='l2')

In [3]:
data['Category'] = pd.Series(data['Category'], dtype="category")

In [4]:
from sklearn.cross_validation import train_test_split
y = data['CatNum']
# X = data[[cat for cat in data.columns if 'Day_' in cat or 'PdD_' in cat]+['X','Y']]
# X = data[['X','Y']]
X = data[['X','Y','DayNum','PdDNum']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [170]:
def dnn_relu(X,y):
    layers = skflow.ops.dnn(X, [500,500,500], tf.nn.relu)
    return skflow.models.logistic_regression(layers, y)

def exp_decay(global_step):
    return tf.train.exponential_decay(global_step,
                                      learning_rate=0.1,                                      
                                      decay_steps=2,
                                      decay_rate=0.001)

def run_NN(X_train, y_train, X_test, y_test, steps=500, learn_rate=1e-2):
    classifier = skflow.TensorFlowEstimator(model_fn=dnn_relu,
                                            n_classes=39,
                                            batch_size=128,
                                            learning_rate=learn_rate,
                                            steps=steps)
    classifier.fit(X_train, y_train)
    return classifier

In [172]:
classifier = run_NN(X_train, y_train, X_test, y_test, steps=10000, learn_rate=1e-2)
pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)
print "Training:", metrics.accuracy_score(pred_train, y_train)
print "Testing:", metrics.accuracy_score(pred_test, y_test)

Step #1, avg. loss: 4.69659
Step #1001, avg. loss: 2.74758
Step #2001, avg. loss: 2.67331
Step #3001, avg. loss: 2.66698
Step #4001, avg. loss: 2.66476
Step #5001, avg. loss: 2.65872
Step #6001, epoch #1, avg. loss: 2.65889
Step #7001, epoch #1, avg. loss: 2.65911
Step #8001, epoch #1, avg. loss: 2.64910
Step #9001, epoch #1, avg. loss: 2.64756
Training: 0.210849625377
Training: 0.21129206765


In [173]:
pred_test_labels = [cat_dict['Category'][p] for p in pred_test]

In [ ]:
#SVM

from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
predict = svc(X_test)

print "Training:", metrics.accuracy_score(pred_train, y_train)
print "Testing:", metrics.accuracy_score(pred_test, y_test)